<a href="https://colab.research.google.com/github/A-n-a-s-t-a-s-i-i-a/Python-for-DA-projects/blob/main/Portfolio_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, shapiro, kendalltau, spearmanr, normaltest, f_oneway, mannwhitneyu, ttest_ind, chi2_contingency
import statsmodels.api as sm
import numpy as np

auth.authenticate_user()

client = bigquery.Client(project="data-analytics-mate")

query = """
with session_info as (
 select
   s.date,
   s.ga_session_id,
   sp.country,
   sp.device,
   sp.continent,
   sp.channel,
   ab.test,
   ab.test_group
 from `DA.ab_test` ab
 join `DA.session` s
 on ab.ga_session_id = s.ga_session_id
 join `DA.session_params` sp
 on ab.ga_session_id = sp.ga_session_id
),
session_with_orders as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct o.ga_session_id) as order_cnt
from `DA.order` o
join session_info
on o.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group
),
events as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 ep.event_name,
 count(ep.ga_session_id) as event_cnt
from `DA.event_params` ep
join session_info
on ep.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 ep.event_name
),
session as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct session_info.ga_session_id) as session_cnt
from session_info
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group
),
account as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct acs.ga_session_id) as account_cnt
from `DA.account_session` acs
join session_info
on acs.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group
)


select
 session_with_orders.date,
 session_with_orders.country,
 session_with_orders.device,
 session_with_orders.continent,
 session_with_orders.channel,
 session_with_orders.test,
 session_with_orders.test_group,
 'session with orders' as event_name,
 session_with_orders.order_cnt as value
from session_with_orders
union all
select
 events.date,
 events.country,
 events.device,
 events.continent,
 events.channel,
 events.test,
 events.test_group,
 event_name,
 event_cnt as value
from events
union all
select
 session.date,
 session.country,
 session.device,
 session.continent,
 session.channel,
 session.test,
 session.test_group,
 'session' as event_name,
 session_cnt as value
from session
union all
select
 account.date,
 account.country,
 account.device,
 account.continent,
 account.channel,
 account.test,
 account.test_group,
 'account' as event_name,
 account_cnt as value
from account
"""

query_job = client.query(query)
results = query_job.result()


df = results.to_dataframe()

df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,account,1


In [ ]:
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "account"]

results_stat_s = []

for test_number in df["test"].unique():
    df_test = df[df["test"] == test_number]

    for metric in metrics:
        df_event = df_test[df_test["event_name"].isin(["session", metric])]

        agg = (
            df_event
            .groupby(["test_group", "event_name"])["value"]
            .sum()
            .unstack(fill_value=0)
            .reset_index()
        )

        control_group = agg[agg["test_group"] == 1]
        test_group = agg[agg["test_group"] == 2]

        events = np.array([test_group[metric].values[0], control_group[metric].values[0]])
        sessions = np.array([test_group["session"].values[0], control_group["session"].values[0]])

        z_stat, p_value = sm.stats.proportions_ztest(events, sessions)

        results_stat_s.append({
            "test_number": test_number,
            "metric": metric,
            "denominator": "session",
            "test_group_event_count": events[0],
            "test_group_sessions": sessions[0],
            "test_group_conversion": events[0] / sessions[0],
            "control_group_event_count": events[1],
            "control_group_sessions": sessions[1],
            "control_group_conversion": events[1] / sessions[1],
            "metric_change": ((events[0] * sessions[1]) / (events[1] * sessions[0]) - 1) * 100,
            "z_stat": z_stat,
            "p_value": p_value,
            "significant": True if p_value < 0.05 else False
        })

result_df = pd.DataFrame(results_stat_s)
print(result_df)

result_df.to_excel("portfolio_project_2_results.xlsx", index=False)

    test_number             metric denominator  test_group_event_count  \
0             2   add_payment_info     session                    2409   
1             2  add_shipping_info     session                    3510   
2             2     begin_checkout     session                    4313   
3             2            account     session                    4184   
4             1   add_payment_info     session                    2229   
5             1  add_shipping_info     session                    3221   
6             1     begin_checkout     session                    4021   
7             1            account     session                    3681   
8             4   add_payment_info     session                    3601   
9             4  add_shipping_info     session                    4956   
10            4     begin_checkout     session                   12267   
11            4            account     session                    8687   
12            3   add_payment_info    

## Посилання на дашборд в [Tableau](https://public.tableau.com/app/profile/anastasiia.paziuka/viz/ABtesting_17472262681660/ABtest)

## Посилання на [файл](https://docs.google.com/spreadsheets/d/121nTfNE8goGWgsbYKPnaxnIVOIF03Gk9/edit?gid=122195585#gid=122195585) з результатами розрахунків